<a href="https://colab.research.google.com/github/fralfaro/MAT281_2022/blob/main/labs/lab_07/lab_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°07


<a id='p1'></a>
## I.- Problema 01


<img src="https://www.cardrates.com/wp-content/uploads/2020/08/shutterstock_576998230.jpg?1" width="480" height="360" align="center"/>


El conjunto de datos se denomina `creditcard.csv` y consta de varias columnas con información acerca del fraude de tarjetas de crédito, en donde la columna **Class** corresponde a: 0 si no es un fraude y 1 si es un fraude.

En este ejercicio se trabajará el problemas de  clases desbalancedas. Veamos las primeras cinco filas dle conjunto de datos:

In [33]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [34]:
# cargar datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/labs/lab_07/data/creditcard.csv",sep=';')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
2,10.0,1.249999,-1.221637,0.383930,-1.234899,-1.485419,-0.753230,-0.689405,-0.227487,-2.094011,...,-0.231809,-0.483285,0.084668,0.392831,0.161135,-0.354990,0.026416,0.042422,121.50,0
3,10.0,0.384978,0.616109,-0.874300,-0.094019,2.924584,3.317027,0.470455,0.538247,-0.558895,...,0.049924,0.238422,0.009130,0.996710,-0.767315,-0.492208,0.042472,-0.054337,9.99,0
4,12.0,1.103215,-0.040296,1.267332,1.289091,-0.735997,0.288069,-0.586057,0.189380,0.782333,...,-0.024612,0.196002,0.013802,0.103758,0.364298,-0.382261,0.092809,0.037051,12.99,0


Analicemos el total de fraudes respecto a los casos que nos son fraudes:


In [35]:
# calcular proporciones
df_count = pd.DataFrame()
df_count["fraude"] =["no","si"]
df_count["total"] = df["Class"].value_counts() 
df_count["porcentaje"] = 100*df_count["total"] /df_count["total"] .sum()

df_count

,fraude,total,porcentaje
0,no,50000,99.025588
1,si,492,0.974412


Se observa que menos del 1% corresponde a registros frudulentos. La pregunta que surgen son:

* ¿ Cómo deben ser el conjunto de entrenamiento y de testeo?
* ¿ Qué modelos ocupar?
* ¿ Qué métricas ocupar?

Por ejemplo, analicemos el modelos de regresión logística y apliquemos el procedimiento estándar:

In [36]:
# datos 
y = df.Class
X = df.drop('Class', axis=1)

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)


# Creando el modelo
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# predecir
lr_pred = lr.predict(X_test)

# calcular accuracy
accuracy_score(y_test, lr_pred)

0.9961181969420898

En general el modelo tiene un **accuracy** del 99,9%, es decir, un podría suponer que el modelo predice casi perfectamente, pero eso esta lejos de ser así.  Para ver por qué es necesario seguir los siguientes pasos:

### 1. Cambiar la métrica de rendimiento

El primer paso es comparar con distintas métricas, para eso ocupemos las 4 métricas clásicas abordadas en el curso:
* accuracy
* precision
* recall
* f-score

En este punto deberá poner las métricas correspondientes y comentar sus resultados.

In [37]:
# metrics
y_true =  list(y_test)
y_pred = list(lr.predict(X_test))

#Guardamos los datos para la comparativa final
accuracy1=accuracy_score(y_test, lr_pred)
recall1=recall_score(y_test, lr_pred)
precision1=precision_score(y_test, lr_pred)
fscore1=f1_score(y_test, lr_pred)

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_test, lr_pred))
print('recall:     ',recall_score(y_test, lr_pred))
print('precision:  ',precision_score(y_test, lr_pred))
print('f-score:    ',f1_score(y_test, lr_pred))
print("")


Matriz de confusion:
 
[[12471    16]
 [   33   103]]

Metricas:
 
accuracy:    0.9961181969420898
recall:      0.7573529411764706
precision:   0.865546218487395
f-score:     0.807843137254902



Como lo visto anteriormente, el accuracy que mide es muy bueno. Sin embargo, podemos ver que las otras medidas van distanciándose del 1. No obstante, sigue siendo un modelo que predice bien el comportamiento de la muestra, pero nosotros queremos ver los casos fraudulentos, y al haber tantos casos correctos, no es un muy bueno modelo para reconocer lo que se desea.

### 2. Cambiar algoritmo

El segundo paso es comparar con distintos modelos. Debe tener en cuenta que el modelo ocupaod resuelva el problema supervisado de clasificación.

En este punto deberá ajustar un modelo de **random forest**, aplicar las métricas y comparar con el modelo de regresión logística.

In [38]:
# train model

rfc =  RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [39]:
# metrics

y_true =  list(y_test)
y_pred = list(rfc.predict(X_test)) # predicciones con random forest

#Guardamos los datos para la comparativa final
accuracy2=accuracy_score(y_true, y_pred)
recall2=recall_score(y_true, y_pred)
precision2=precision_score(y_true, y_pred)
fscore2=f1_score(y_true, y_pred)

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12484     3]
 [   24   112]]

Metricas:
 
accuracy:    0.9978610472946209
recall:      0.8235294117647058
precision:   0.9739130434782609
f-score:     0.8924302788844622



Notemos que el nuevo modelo tiene aún más precisión y pareciera acercarse más a la realidad que el anterior. Pero, por el funcionamiento de un RandomForest, es muy poco probable que los datos que haya tomado sean parte de los datos fraudulentos, por lo que, al igual que el anterior, a pesar de tener datos muy cercanos a 1, este no sirve para el objetivo buscado.

### 3. Técnicas de remuestreo: sobremuestreo de clase minoritaria

El tercer paso es ocupar ténicas de remuestreo, pero sobre la clase minoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase minoritaria a la clase mayoritaria.

In [40]:
from sklearn.utils import resample

# concatenar el conjunto de entrenamiento
X = pd.concat([X_train, y_train], axis=1)

# separar las clases
not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

# remuestrear  clase minoritaria
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# recombinar resultados
upsampled = pd.concat([not_fraud, fraud_upsampled])

# chequear el número de elementos por clases
upsampled.Class.value_counts()

0    37513
1    37513
Name: Class, dtype: int64

In [41]:
# datos de entrenamiento sobre-balanceados
y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [42]:
upsampled = LogisticRegression(solver='liblinear') # algoritmo de regresion logistica, usando lo visto al inicio del lab
upsampled.fit(X_train,y_train)

# metrics

y_true =  list(y_test)
y_pred = list(upsampled.predict(X_test))

#Guardamos los datos para la comparativa final
accuracy3=accuracy_score(y_true, y_pred)
recall3=recall_score(y_true, y_pred)
precision3=precision_score(y_true, y_pred)
fscore3=f1_score(y_true, y_pred)

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12200   287]
 [   12   124]]

Metricas:
 
accuracy:    0.976313079299691
recall:      0.9117647058823529
precision:   0.30170316301703165
f-score:     0.45338208409506403



El sobremuestreo hace que la cantidad de datos minoritaria (los fraudes) alcancen la cantidad de datos mayoritaria (los no fraudulentos), por lo que tiene la ventaja de no tener pérdida de información, sirve para comparativas de datos desbalanceados, y de desventaja, se puede tener una mala visión de la realidad de los datos por la réplica de datos minoritarios.

A efectos de nuestro ejercicio, resulta ser un buen modelo. Se puede notar que la precisión y el f-score bajan mucho respecto a los otros modelos, lo que indica que los datos minoritarios están causando problemas, y es un buen modelo para buscar fraudes.

### 4. Técnicas de remuestreo - Ejemplo de clase mayoritaria

El cuarto paso es ocupar ténicas de remuestreo, pero sobre la clase mayoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase mayoritaria  a la clase minoritaria.

In [43]:
# remuestreo clase mayoritaria
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# recombinar resultados
downsampled = pd.concat([not_fraud_downsampled, fraud])

# chequear el número de elementos por clases
downsampled.Class.value_counts()

0    356
1    356
Name: Class, dtype: int64

In [44]:
# datos de entrenamiento sub-balanceados

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, |justifique las ventajas y desventjas de este procedimiento.

In [45]:
undersampled = LogisticRegression(solver='liblinear') # algoritmo de regresion logistic
undersampled.fit(X_train,y_train)
# metrics

y_true =  list(y_test)
y_pred = list(undersampled.predict(X_test))

#Guardamos los datos para la comparativa final
accuracy4=accuracy_score(y_true, y_pred)
recall4=recall_score(y_true, y_pred)
precision4=precision_score(y_true, y_pred)
fscore4=f1_score(y_true, y_pred)

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12216   271]
 [   16   120]]

Metricas:
 
accuracy:    0.9772637249465261
recall:      0.8823529411764706
precision:   0.3069053708439898
f-score:     0.45540796963946867



El submuestreo hace que los datos mayoritarios se reduzcan hasta alcanzar los minoritarios. Esto tiene como clara desventaja una pérdida de información, y de ventaja que, de ser muchos datos, el tiempo de ejecución mejora mucho.

A efectos de nuestro modelo, podemos ver lo mismo que en el modelo anterior, donde la precisión y el f-score bajan mucho respecto a modelos anteriores (sin contar el sobremuestreo), y sirve para medir datos fraudulentos por la misma razón que en el sobremuestreo.

### 5. Conclusiones

Para finalizar el laboratorio, debe realizar un análisis comparativo con los disintos resultados obtenidos  en los pasos 1-4. Saque sus propias conclusiones del caso.

In [46]:
Datos = {'n_modelo': ['1','2','3','4'],
          'accuracy': [accuracy1,accuracy2,accuracy3,accuracy4],
          'recall': [recall1,recall2,recall3,recall4],
          'precision': [precision1,precision2,precision3,precision4],
          'f-score': [fscore1,fscore2,fscore3,fscore4],
}

AnalisisDeDatos=pd.DataFrame(Datos)
AnalisisDeDatos

,n_modelo,accuracy,recall,precision,f-score
0,1,0.996118,0.757353,0.865546,0.807843
1,2,0.997861,0.823529,0.973913,0.892430
2,3,0.976313,0.911765,0.301703,0.453382
3,4,0.977264,0.882353,0.306905,0.455408


Podemos comenzara  concluir.
Ya vimos que, a pesar de que el primer y segundo modelo son muy buenos para predecir datos en lo general, estos no sirven, a efectos prácticos, en lo que nosotros buscamos, pues la cantidad de datos de interés (los fraudes) son tan despreciables, que el modelo es poco probable que los considere.

Con esto, notemos que el tercer y cuarto modelo presentan datos similares en cuanto a precision y f-score, que están, además, alejados del 1, y un recall muy alto (junto con un accuracy). Esto es de suma importancia, un recall y un accuracy alto significan que el modelo, la mayor parte del tiempo, resulta fidedigno en cuanto a los datos verdaderos, y al tener un f1 y una precisión baja, implica que los datos fraudulentos hacen ruido en la muestra, y por ende, son buenos modelos para identificar casos fraudulentos, pues fueron entrenados con estos.

Además, el tercer modelo pareciera ser el mejor, pues, además, no existe pérdida de información como en el cuarto.